# Geohash + CID


In [ ]:
#!pip install pygeohash

In [2]:
import pygeohash as pgh
##load geojson
import geopandas as gpd
import pandas as pd
import os
import sys
sys.path.append("../")

Can only import/use native python functions.


In [3]:
from geohashtree.geohashtree import LiteTreeOffset, LiteTreeCID, FullTreeFile
from geohashtree.filesystem import ipfs_add_feature,ipfs_add_index_folder

which: no ipfs in (/gpfs/data1/oshangp/easier/geohash-cid/py39/bin:/opt/TurboVNC/bin:/opt/TurboVNC/bin:/home/liuz/.vscode-server/bin/0ee08df0cf4527e40edc9aa28f4b5bd38bbff2b2/bin/remote-cli:/opt/TurboVNC/bin:/gpfs/data1/oshangp/easier/geohash-cid/py39/bin:/apps/python/3.7/anaconda/condabin:/opt/TurboVNC/bin:/usr/local/bin:/usr/bin:/opt/puppetlabs/bin:/usr/local/sbin:/usr/sbin:/home/liuz/.local/bin:/home/liuz/bin:/gpfs/data1/oshangp/easier/tools/gopath/bin:/home/liuz/.local/bin:/home/liuz/bin)


IPFS path: /gpfs/data1/oshangp/easier/textile/kubo/ipfs


In [8]:
asset = "us_places_gh_sorted"
geojson_path = f"../data/overture/{asset}.geojson"
index_path = f"../data/geohash_offset_{asset}_d3/index"

## Detached tree with offset
### Index Preparation
using one large file + lite index tree with offset

In [9]:
offsettree = LiteTreeOffset()

Index Mode: offline


In [10]:
offsettree.add_from_geojson(geojson_path,precision=3)
offsettree.generate_tree_index(index_path)

### Onloading

In [11]:
index_cid = ipfs_add_index_folder(index_path)

In [12]:
index_cid

'bafybeie5fs3x7wke4b7vnrk6wfgae43j6rlxg4airjefqd5dwey2xkrdbu'

In [ ]:
index_cid = "bafybeib6ex2onm2wk5wxrdfe73l4m3la3c35nr5f4b4trbtidqaofduqiu"

In [ ]:
ipfs_add_feature(geojson_path)

In [ ]:
cid = "bafybeia7jlkyzt22qgh66felldr322l5c7jdvm4xspr222f4usnd2ttbze"

In [ ]:
##debugging
# import subprocess

# of,lg = (6266126, 4005)

# result = subprocess.run(['ipfs', 'cat', cid, "-o",str(of),"-l",str(lg)], stdout=subprocess.PIPE, check=True, text=True)

## Spatial query
Perform a spatial query and return a list of geohashes

In [ ]:
geohashes = ['dqcjy',
 'dqcjz',
 'dqcjw',
 'dqcjv',
 'dqcjx',
 'dqcjq',
 'dqcjt',
 'dqcju',
 'dqcjr',
 'dqcjm',
 'dqcjs',
 'dqcjg',
 'dqcjk',
 'dqcnn',
 'dqcnq',
 'dqcnp',
 'dqcnj',
 'dqcnr',
 'dqcnm',
 'dqcnh',
 'dqcnk',
 'dqcmb',
 'dqcm8',
 'dqcm2',
 'dqcq0']

In [ ]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

## Feature Retrieval

In [ ]:
offsettree.query(geohashes,index_cid)

In [ ]:
offsettree.retrieve(geohashes,index_cid)

## Detached tree with CID

In [13]:
import sys
sys.path.append("../")
from geohashtree.geohashtree import LiteTreeCID,append_h3_to_dataframe
from geohashtree.filesystem import ipfs_add_index_folder,ipfs_get_index_folder

In [7]:
#Entire workflow for LiteTreeCID

asset = "us_places"
raw_path = f"../data/overture/{asset}.parquet"
index_path = f"../data/geohash_cid_{asset}_parq_d5/index"
pieces_path = f"../data/geohash_cid_{asset}_parq_d5/pieces"

In [22]:
#debugging
gdf = gpd.read_parquet(raw_path)

In [8]:
cidtree = LiteTreeCID()
cidtree.file_format='parquet'

In [ ]:
cidtree.add_from_parquet(raw_path,pieces_path,precision=5)

  geohash                                       parquet_path
0   9kxpz  ../data/geohash_cid_us_places_parq_d5/pieces/9...
1   9kxvr  ../data/geohash_cid_us_places_parq_d5/pieces/9...
2   9kxvv  ../data/geohash_cid_us_places_parq_d5/pieces/9...
3   9kxwy  ../data/geohash_cid_us_places_parq_d5/pieces/9...
4   9kxx7  ../data/geohash_cid_us_places_parq_d5/pieces/9...


In [ ]:
cidtree.generate_tree_index(index_path)

In [ ]:
cidtree.upload_parquet_files(pieces_path)

In [ ]:
index_cid = ipfs_add_index_folder(index_path)

In [16]:
index_cid

'bafybeicttshfwol4g5qsepp3pnukmjwl6tols2i2xl2sxo2svgg2d2mq4y'

In [ ]:
ipfs_get_index_folder('bafybeihfj2qf7c4lqbwdle7e6vqclexrvc4xf3evaa77z43gu5tbtrj53e','../data/test/geohash_cid_dc_parq')

In [ ]:
local_index_path = '../data/test/geohash_cid_dc_parq'

In [ ]:
cidtree.file_format

In [ ]:
cidtree.

In [ ]:
import glob
piece_files = glob.glob(pieces_path+"/*.geojson")

In [ ]:
ipfs_add_feature(piece_files)

In [ ]:
geojson_path

In [ ]:
geohashes = ['dqcn', 'dqcj']

In [ ]:
cids = cidtree.query(geohashes,local_index_path)

In [ ]:
len(cids)

In [ ]:
cidtree.file_format = 'parquet'

In [ ]:
ret = cidtree.retrieve(geohashes,local_index_path)

In [ ]:
ret

In [ ]:
from geohashtree.geohashtree import FullTreeFile

## Attached tree with file

In [ ]:
#Entire workflow for FullTreeFile

asset = "us_places_100k"
geojson_path = f"../data/overture/{asset}.geojson"
index_path = f"../data/geohash_full_{asset}/index"

In [ ]:
fulltree = FullTreeFile()
fulltree.add_from_geojson(geojson_path)
fulltree.generate_tree_index(index_path)
ipfs_add_index_folder(index_path)

In [ ]:
geohashes = ['dqcnn', 'dqcjz', 'dqcjw', 'dqcjv', 'dqcnp', 'dqcjx', 'dqcjt', 'dqcnj']

In [ ]:
cid = "bafybeiez73pdnpevvoptzft54zmljj54di2pp44slk67wfeo7uz5hdbqoe/d/dq/dqc/dqcn/dqcnn"

In [ ]:
import os
os.path.join(cid,"dqcnn0.geojson")

In [ ]:
index_cid = "bafybeiez73pdnpevvoptzft54zmljj54di2pp44slk67wfeo7uz5hdbqoe"

In [ ]:
results = fulltree.query(geohashes,index_cid)

In [ ]:
len(results)

In [ ]:
results

In [ ]:
ret = fulltree.retrieve(geohashes,index_cid)

In [ ]:
ret